# Python script to update a column in a table in Oracle db

In [1]:
# Importing library packages
import cx_Oracle # python package for oracle db connection 
import pandas as pd # python package for dataframes

In [2]:
# DB connection setup
testdb_tnsdsn = cx_Oracle.makedsn('<Hostname>', '<Port>', service_name='<Service name>') 
conn_v = cx_Oracle.connect(user='<Username>', password='<Password>'.encode(), dsn=testdb_tnsdsn)

In [3]:
# Declaring variables
table_name = 'ID_TEST' # Give the name of the table in the database
column_name = 'ACCT_ID' # Change it with the column name based on which we will be querying
update_columnname = 'DESCRIPTION' # column name to look into inside the table
update_columndesc = 'ID present in excel sheet' # description that will be updated in the table column
sheet_name = 'sheet1' # Change the name of the sheet according to the requirement based on the excel sheet
srcxlsx_df = pd.ExcelFile(r'excel_columns.xlsx').parse('{:s}'.format(sheet_name)) # Reading data from excel sheet and storing it into a variable

In [4]:
print(srcxlsx_df) # Prints the account ids listed in the excel sheet

   ACCT_ID
0    23456
1    13456
2    67890


In [5]:
dbtbl_bfrupdt= pd.read_sql_query("select * from TEST_DB.ID_TEST",conn_v)
dbtbl_bfrupdt #Displays records in the table (before making any changes)

,ACCT_ID,DESCRIPTION
0,12345,None
1,32345,None
2,23456,None
3,13456,None
4,34567,None
5,45678,ID found in excel
6,56789,None
7,67890,None


In [6]:
acctcnt_updtd = 0  # creating a variable with value as zero. whenever there is an update it will increase the count
c = conn_v.cursor() # Creating a cursor. It will be used later on for updating the column of a table in db
for x in srcxlsx_df['ACCT_ID']:
    db_column= pd.read_sql_query("select {:s} from {:s} where {:s} = '{:d}' ".format(column_name,table_name,column_name,x),conn_v)
    db_column
    if db_column.empty is False:
        db_value = db_column['ACCT_ID'][0]
        c.execute("update {:s} set {:s}= '{:s}' where {:s}='{:s}'".format(table_name,update_columnname,update_columndesc,column_name,db_value))
        acctcnt_updtd += 1
        conn_v.commit()
        
    else:
        print("Check table in database.")  # It will be printed if there source sheet is empty.
print("Total updated account ids count: {:d}".format(acctcnt_updtd)) # Printing the total updated accounts count

Total updated account ids count: 3


In [7]:
dbtbl_aftrupdt= pd.read_sql_query("select * from TEST_DB.ID_TEST",conn_v)
dbtbl_aftrupdt #Displays records in the table (after making changes)

,ACCT_ID,DESCRIPTION
0,12345,None
1,32345,None
2,23456,ID present in excel sheet
3,13456,ID present in excel sheet
4,34567,None
5,45678,ID found in excel
6,56789,None
7,67890,ID present in excel sheet


In [8]:
conn_v.close() # Close the database connection 